# 1.1

In [16]:
# Global alignment (Needleman–Wunsch) for:
# S1 = ACGTACGTAC
# S2 = ACGTACGATC
#
# Scoring:
#   match = +1
#   mismatch = -1
#   gap (indel) = -1

MATCH_SCORE = 1
MISMATCH_SCORE = -1
GAP_SCORE = -1
S1 = "CACAGCATTT"
S2 = "TACGAGGAGT"


def score(a, b):
    """Return match/mismatch score for two characters."""
    return MATCH_SCORE if a == b else MISMATCH_SCORE


def needleman_wunsch_dp(s1, s2):
    n = len(s1)
    m = len(s2)

    # dp[i][j] = best score for aligning s1[:i] with s2[:j]
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    # Initialize first row and column (global alignment)
    for i in range(1, n + 1):
        dp[i][0] = i * GAP_SCORE
    for j in range(1, m + 1):
        dp[0][j] = j * GAP_SCORE

    # Fill DP table
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            diag = dp[i - 1][j - 1] + score(s1[i - 1], s2[j - 1])
            up = dp[i - 1][j] + GAP_SCORE      # gap in s2
            left = dp[i][j - 1] + GAP_SCORE    # gap in s1
            dp[i][j] = max(diag, up, left)

    return dp


def print_dp_table(dp, s1, s2):
    """Nicely print the DP table with sequence labels."""
    n = len(s1)
    m = len(s2)

    print("Global alignment DP table:")
    print()
    # Header row
    header = ["    "]  # top-left corner
    header.append("   ")  # for the leading "-"
    for ch in s2:
        header.append(f"{ch:>4}")
    print("".join(header))

    # First row (i = 0)
    row_str = ["   -"]
    for j in range(m + 1):
        row_str.append(f"{dp[0][j]:>4}")
    print("".join(row_str))

    # Remaining rows
    for i in range(1, n + 1):
        row_str = [f"   {s1[i-1]}"]
        for j in range(m + 1):
            row_str.append(f"{dp[i][j]:>4}")
        print("".join(row_str))
    print()


def traceback_all(dp, s1, s2, max_alignments=2):
    """
    Collect up to max_alignments distinct optimal global alignments
    by exploring all optimal traceback paths.
    Returns a list of dicts, each with:
      {
        "aligned_s1": ...,
        "aligned_s2": ...,
        "path": [(i,j), ...]   # traceback matrix coordinates from start to end
      }
    """
    n = len(s1)
    m = len(s2)
    alignments = []
    seen = set()  # to avoid duplicates based on (aligned_s1, aligned_s2)

    def dfs(i, j, aligned1_rev, aligned2_rev, path_rev):
        # Stop if we already have enough alignments
        if len(alignments) >= max_alignments:
            return

        # Base case: reached the origin (0,0)
        if i == 0 and j == 0:
            aligned1 = "".join(reversed(aligned1_rev))
            aligned2 = "".join(reversed(aligned2_rev))
            key = (aligned1, aligned2)
            if key not in seen:
                seen.add(key)
                path = list(reversed(path_rev))
                alignments.append({
                    "aligned_s1": aligned1,
                    "aligned_s2": aligned2,
                    "path": path
                })
            return

        current_score = dp[i][j]

        # We record this cell in the path
        new_path_rev = path_rev + [(i, j)]

        # Option 1: Diagonal (match/mismatch)
        if i > 0 and j > 0:
            diag_score = dp[i - 1][j - 1] + score(s1[i - 1], s2[j - 1])
            if diag_score == current_score:
                dfs(i - 1, j - 1,
                    aligned1_rev + [s1[i - 1]],
                    aligned2_rev + [s2[j - 1]],
                    new_path_rev)

        # Option 2: Up (gap in s2)
        if i > 0:
            up_score = dp[i - 1][j] + GAP_SCORE
            if up_score == current_score:
                dfs(i - 1, j,
                    aligned1_rev + [s1[i - 1]],
                    aligned2_rev + ['-'],
                    new_path_rev)

        # Option 3: Left (gap in s1)
        if j > 0:
            left_score = dp[i][j - 1] + GAP_SCORE
            if left_score == current_score:
                dfs(i, j - 1,
                    aligned1_rev + ['-'],
                    aligned2_rev + [s2[j - 1]],
                    new_path_rev)

    # Start DFS from bottom-right cell
    dfs(n, m, [], [], [])
    return alignments


def main():
    print("S1:", S1)
    print("S2:", S2)
    print()

    # 1. Compute DP table
    dp = needleman_wunsch_dp(S1, S2)

    # 2. Print DP table
    print_dp_table(dp, S1, S2)

    # 3. Traceback to get two distinct optimal global alignments
    results = traceback_all(dp, S1, S2, max_alignments=2)

    print("Optimal global alignment score:", dp[len(S1)][len(S2)])
    print()

    for idx, aln in enumerate(results, start=1):
        print(f"Alignment #{idx}:")
        print("Aligned S1:", aln["aligned_s1"])
        print("Aligned S2:", aln["aligned_s2"])
        print("Traceback path (i,j) from start to end:")
        print(aln["path"])
        print()


if __name__ == "__main__":
    main()


S1: CACAGCATTT
S2: TACGAGGAGT

Global alignment DP table:

          T   A   C   G   A   G   G   A   G   T
   -   0  -1  -2  -3  -4  -5  -6  -7  -8  -9 -10
   C  -1  -1  -2  -1  -2  -3  -4  -5  -6  -7  -8
   A  -2  -2   0  -1  -2  -1  -2  -3  -4  -5  -6
   C  -3  -3  -1   1   0  -1  -2  -3  -4  -5  -6
   A  -4  -4  -2   0   0   1   0  -1  -2  -3  -4
   G  -5  -5  -3  -1   1   0   2   1   0  -1  -2
   C  -6  -6  -4  -2   0   0   1   1   0  -1  -2
   A  -7  -7  -5  -3  -1   1   0   0   2   1   0
   T  -8  -6  -6  -4  -2   0   0  -1   1   1   2
   T  -9  -7  -7  -5  -3  -1  -1  -1   0   0   2
   T -10  -8  -8  -6  -4  -2  -2  -2  -1  -1   1

Optimal global alignment score: 1

Alignment #1:
Aligned S1: CAC-AGCATTT
Aligned S2: TACGAGGA-GT
Traceback path (i,j) from start to end:
[(1, 1), (2, 2), (3, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 8), (9, 9), (10, 10)]

Alignment #2:
Aligned S1: CAC-AGCATTT
Aligned S2: TACGAGGAG-T
Traceback path (i,j) from start to end:
[(1, 1), (2, 2), (3, 3

1.1.3

In [17]:
# End-space-free alignment DP for:
# S1 and S2 under scoring:
#   match = +1
#   mismatch = -1
#   gap (indel) = -1
#
# End-space-free definition here:
# - No penalty for gaps at the BEGINNING or END of either sequence.
# - Internal gaps are penalized as in global alignment.
# - DP first row and first column are initialized to 0.
# - The optimal score is the maximum value anywhere in the DP table.
# - Traceback starts from that max cell and stops when we reach row 0 or col 0
#   (free unaligned prefixes).

MATCH_SCORE = 1
MISMATCH_SCORE = -1
GAP_SCORE = -1

# <<< PUT YOUR SEQUENCES HERE >>>
S1 = "CACAGCATTT"
S2 = "TACGAGGAGT"


def score(a, b):
    return MATCH_SCORE if a == b else MISMATCH_SCORE


def end_space_free_dp(s1, s2):
    n = len(s1)
    m = len(s2)

    # dp[i][j] = best score for an end-space-free alignment ending at s1[:i], s2[:j]
    dp = [[0] * (m + 1) for _ in range(n + 1)]

    # First row and first column are all 0 (free leading gaps).
    # So no extra initialization needed, they are already 0.

    # Fill DP
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            diag = dp[i - 1][j - 1] + score(s1[i - 1], s2[j - 1])
            up = dp[i - 1][j] + GAP_SCORE      # gap in s2
            left = dp[i][j - 1] + GAP_SCORE    # gap in s1
            dp[i][j] = max(diag, up, left)

    # Find the cell with the maximum score anywhere in the table
    best_score = float("-inf")
    best_pos = (0, 0)
    for i in range(n + 1):
        for j in range(m + 1):
            if dp[i][j] > best_score:
                best_score = dp[i][j]
                best_pos = (i, j)

    return dp, best_score, best_pos


def print_dp_table(dp, s1, s2, title="End-space-free alignment DP table:"):
    n = len(s1)
    m = len(s2)

    print(title)
    print()
    # Header row
    header = ["    ", "   "]  # corner + leading "-" column header
    for ch in s2:
        header.append(f"{ch:>4}")
    print("".join(header))

    # First row (i = 0, s1 prefix is empty)
    row_str = ["   -"]
    for j in range(m + 1):
        row_str.append(f"{dp[0][j]:>4}")
    print("".join(row_str))

    # Remaining rows
    for i in range(1, n + 1):
        row_str = [f"   {s1[i-1]}"]
        for j in range(m + 1):
            row_str.append(f"{dp[i][j]:>4}")
        print("".join(row_str))
    print()


def traceback_end_space_free(dp, s1, s2, start_i, start_j):
    """
    Traceback from (start_i, start_j) until we reach row 0 or col 0.
    Returns aligned_s1, aligned_s2, path (list of (i,j) from start to end).
    """
    i, j = start_i, start_j
    aligned1_rev = []
    aligned2_rev = []
    path_rev = []

    while i > 0 and j > 0:
        path_rev.append((i, j))
        current = dp[i][j]
        # Prefer diagonal, then up, then left (this gives one valid optimal path)
        if current == dp[i - 1][j - 1] + score(s1[i - 1], s2[j - 1]):
            aligned1_rev.append(s1[i - 1])
            aligned2_rev.append(s2[j - 1])
            i -= 1
            j -= 1
        elif current == dp[i - 1][j] + GAP_SCORE:
            aligned1_rev.append(s1[i - 1])
            aligned2_rev.append('-')
            i -= 1
        else:  # current == dp[i][j - 1] + GAP_SCORE
            aligned1_rev.append('-')
            aligned2_rev.append(s2[j - 1])
            j -= 1

    # We stop when i == 0 or j == 0 because prefixes are free (no penalties).
    # We do NOT add leading gaps to the alignment: they are unaligned ends.
    path = list(reversed(path_rev))
    aligned_s1 = "".join(reversed(aligned1_rev))
    aligned_s2 = "".join(reversed(aligned2_rev))

    return aligned_s1, aligned_s2, path


def main():
    print("S1:", S1)
    print("S2:", S2)
    print()

    # 1. Compute DP and best position for end-space-free alignment
    dp, best_score, (bi, bj) = end_space_free_dp(S1, S2)

    # 2. Print DP table
    print_dp_table(dp, S1, S2)

    # 3. Traceback from best cell
    aligned_s1, aligned_s2, path = traceback_end_space_free(dp, S1, S2, bi, bj)

    print("Optimal end-space-free alignment score:", best_score)
    print("Best cell (i,j):", (bi, bj))
    print()

    print("Final end-space-free alignment:")
    print("Aligned S1:", aligned_s1)
    print("Aligned S2:", aligned_s2)
    print()

    print("Traceback path (i,j) from start to end:")
    print(path)


if __name__ == "__main__":
    main()


S1: CACAGCATTT
S2: TACGAGGAGT

End-space-free alignment DP table:

          T   A   C   G   A   G   G   A   G   T
   -   0   0   0   0   0   0   0   0   0   0   0
   C   0  -1  -1   1   0  -1  -1  -1  -1  -1  -1
   A   0  -1   0   0   0   1   0  -1   0  -1  -2
   C   0  -1  -1   1   0   0   0  -1  -1  -1  -2
   A   0  -1   0   0   0   1   0  -1   0  -1  -2
   G   0  -1  -1  -1   1   0   2   1   0   1   0
   C   0  -1  -2   0   0   0   1   1   0   0   0
   A   0  -1   0  -1  -1   1   0   0   2   1   0
   T   0   1   0  -1  -2   0   0  -1   1   1   2
   T   0   1   0  -1  -2  -1  -1  -1   0   0   2
   T   0   1   0  -1  -2  -2  -2  -2  -1  -1   1

Optimal end-space-free alignment score: 2
Best cell (i,j): (5, 6)

Final end-space-free alignment:
Aligned S1: CAC-AG
Aligned S2: TACGAG

Traceback path (i,j) from start to end:
[(1, 1), (2, 2), (3, 3), (3, 4), (4, 5), (5, 6)]
